In [43]:
from neo4j import GraphDatabase

class Neo4jGraphBuilder:
    def __init__(self, uri, user, password, database):
        self.driver = GraphDatabase.driver(uri, auth=(user, password), database=database)

    def close(self):
        self.driver.close()

    def create_node(self, concept):
        with self.driver.session() as session:
            session.run(
                "MERGE (c:Concept {name: $name})",
                name=concept
            )

    def create_relationship(self, node1, node2, relationship):
        with self.driver.session() as session:
            session.run(
                f"MATCH (a:Concept {{name: $node1}}), (b:Concept {{name: $node2}}) "
                f"MERGE (a)-[:{relationship}]->(b)",
                node1=node1,
                node2=node2
            )



In [7]:

import pandas as pd
# Exemple de fonction pour construire le graphe
def build_graph(graph_file):
    # Remplacez par vos informations de connexion
    uri = "bolt://localhost:7687"
    user = "neo4j"
    password = "abdou12345;"
    database = "database"

    # Instanciation de la classe de construction du graphe
    graph_builder = Neo4jGraphBuilder(uri, user, password, database)

    # Lire le fichier CSV avec pandas
    df = pd.read_csv(graph_file, sep="|")  # Assurez-vous que le fichier CSV utilise '|' comme séparateur

    # Parcourir chaque ligne du DataFrame pour créer des nœuds et des relations
    for index, row in df.iterrows():
        node1 = row['node_1']
        node2 = row['node_2']
        relationship = row['edge']

        # Création des nœuds
        graph_builder.create_node(node1)
        graph_builder.create_node(node2)

        # Créer une relation entre node1 et node2
        graph_builder.create_relationship(node1, node2, relationship)

    # Fermer la connexion à la base de données
    graph_builder.close()

In [8]:
# Exemple d'utilisation
graph_file = 'data_output/graph.csv'
# Construire le graphe
build_graph(graph_file)